In [9]:

# import libraries
import numpy as np # for numerical operations
import pandas as pd # for event_log data frame
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 10)
import queue # add FIFO queue data structure
from functools import partial, wraps
import os
import random

import simpy # discrete event simulation environment

In [10]:
'''        
---------------------------------------------------------
set up event tracing of all simulation program events 
controlled by the simulation environment
that is, all timeout and process events that begin with "env."
documentation at:
  https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing
  https://docs.python.org/3/library/functools.html#functools.partial
'''
def trace(env, callback):
     """Replace the ``step()`` method of *env* with a tracing function
     that calls *callbacks* with an events time, priority, ID and its
     instance just before it is processed.
     note: "event" here refers to simulaiton program events

     """
     def get_wrapper(env_step, callback):
         """Generate the wrapper for env.step()."""
         @wraps(env_step)
         def tracing_step():
             """Call *callback* for the next event if one exist before
             calling ``env.step()``."""
             if len(env._queue):
                 t, prio, eid, event = env._queue[0]
                 callback(t, prio, eid, event)
             return env_step()
         return tracing_step

     env.step = get_wrapper(env.step, callback)

def trace_monitor(data, t, prio, eid, event):
     data.append((t, eid, type(event)))

def test_process(env):
     yield env.timeout(1)

'''
---------------------------------------------------------
set up an event log for recording events
as defined for the discrete event simulation
we use a list of tuples for the event log
documentation at:
  https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing
'''     
def event_log_append(env, caseid, time, activity, event_log):
    event_log.append((caseid, time, activity))
    yield env.timeout(0)

In [11]:
### Arrival and in the queue waiting process

def arrival_waiting(env, sim_params, resources, caseid_queue_dinein, caseid_queue_takeout, event_log):
    caseid = 0
    while True:
        inter_arrival_time = round(60 * np.random.exponential(scale=sim_params['ARRIVAL_RATE']))
        print("NEXT ARRIVAL TIME: ", env.now + inter_arrival_time)

        yield env.timeout(inter_arrival_time)
        caseid += 1
        time = env.now
        activity = 'arrival'
        env.process(event_log_append(env, caseid, time, activity, event_log))
        yield env.timeout(0)

        if np.random.random() < sim_params['DINE_IN_PROB']:
            # Dine-in customer
            if caseid_queue_dinein.qsize() < sim_params['MAX_QUEUE_LENGTH']:
                caseid_queue_dinein.put(caseid)
                print("Customer joins dine-in queue caseid =", caseid, 'time =', env.now, 'queue_length =', caseid_queue_dinein.qsize())
                time = env.now
                activity = 'join_queue_dinein'
                env.process(event_log_append(env, caseid, time, activity, event_log))
                env.process(dinein_service_process(env, sim_params, resources, caseid_queue_dinein, event_log))
            else:
                print("Dine-in customer balks caseid =", caseid, 'time =', env.now, 'queue_length =', caseid_queue_dinein.qsize())
                env.process(event_log_append(env, caseid, env.now, 'balk_dinein', event_log))
        else:
            # Takeout customer
            if caseid_queue_takeout.qsize() < sim_params['MAX_QUEUE_LENGTH']:
                caseid_queue_takeout.put(caseid)
                print("Customer joins takeout queue caseid =", caseid, 'time =', env.now, 'queue_length =', caseid_queue_takeout.qsize())
                time = env.now
                activity = 'join_queue_takeout'
                env.process(event_log_append(env, caseid, time, activity, event_log))
                env.process(takeout_service_process(env, sim_params, resources, caseid_queue_takeout, event_log))
            else:
                print("Takeout customer balks caseid =", caseid, 'time =', env.now, 'queue_length =', caseid_queue_takeout.qsize())
                env.process(event_log_append(env, caseid, env.now, 'balk_takeout', event_log))


### Dine-in process

def dinein_service_process(env, sim_params, resources, caseid_queue, event_log):
    with resources['tables'].request() as table, resources['servers'].request() as server:
        caseid = caseid_queue.get()
        patience = random.uniform(sim_params['MIN_PATIENCE'], sim_params['MAX_PATIENCE'])
        
        for item in event_log:
            if item[2] == 'arrival' and item[0] == caseid:
                arrive = item[1]
                break
        
        wait = env.now - arrive
        results = yield table | env.timeout(patience)
        
        if table in results:
            results = yield server | env.timeout(patience)
            
            if server in results:
                print("Customer seated and served caseid =", caseid, 'time =', env.now, 'wait =', wait)
                time = env.now
                activity = 'seated'
                env.process(event_log_append(env, caseid, time, activity, event_log))
                
                activity = 'served'
                env.process(event_log_append(env, caseid, time, activity, event_log))
                
                activity = 'eating'
                yield env.timeout(sim_params['DINEIN_SERVICE_TIME'])
                
                time = env.now
                activity = 'finished_eating'
                env.process(event_log_append(env, caseid, time, activity, event_log))
                print("Customer finished eating caseid =", caseid, 'time =', env.now)
                
                activity = 'paying'
                env.process(event_log_append(env, caseid, time, activity, event_log))
                yield env.timeout(sim_params['PAYMENT_TIME'])
                
                time = env.now
                activity = 'leaving'
                env.process(event_log_append(env, caseid, time, activity, event_log))
                yield env.timeout(0)
            else:
                print("Customer left due to impatience caseid =", caseid, 'time =', env.now)
                time = env.now
                activity = 'impatience'
                env.process(event_log_append(env, caseid, time, activity, event_log))


## Take-out process

def takeout_service_process(env, sim_params, resources, caseid_queue, event_log):
    with resources['cooks'].request() as cook:
        caseid = caseid_queue.get()
        patience = random.uniform(sim_params['MIN_PATIENCE'], sim_params['MAX_PATIENCE'])
        
        for item in event_log:
            if item[2] == 'arrival' and item[0] == caseid:
                arrive = item[1]
                break
        
        wait = env.now - arrive
        results = yield cook | env.timeout(patience)
        
        if cook in results:
            print("Takeout order being prepared caseid =", caseid, 'time =', env.now, 'wait =', wait)
            time = env.now
            activity = 'cooking'
            env.process(event_log_append(env, caseid, time, activity, event_log))
            
            yield env.timeout(sim_params['TAKEOUT_SERVICE_TIME'])
            
            time = env.now
            activity = 'finished_takeout'
            env.process(event_log_append(env, caseid, time, activity, event_log))
            print("Takeout order finished caseid =", caseid, 'time =', env.now)
            
            time = env.now
            activity = 'leaving_takeout'
            env.process(event_log_append(env, caseid, time, activity, event_log))
            yield env.timeout(0)
        else:
            print("Takeout customer left due to impatience caseid =", caseid, 'time =', env.now)
            time = env.now
            activity = 'impatience'
            env.process(event_log_append(env, caseid, time, activity, event_log))

## Cooking process

def cook(env, sim_params, resources, caseid_queue, event_log):
    with resources['cooks'].request() as cook:
        
        print("Cook started cooking at time =", env.now)
        time = env.now
        activity = 'cooking'
        env.process(event_log_append(env, 0, time, activity, event_log))

        results = yield cook

        if cook in results:
            yield env.timeout(sim_params['COOK_TIME'])
            print("Cook finished cooking at time =", env.now)
            yield env.timeout(0)


In [12]:
## Restaurant Simulation Constants

# Resources
NUM_SERVERS = 1  # Number of servers
NUM_COOKS = 2  # Number of cooks in the kitchen
NUM_TABLES = 10  # Number of tables in the restaurant
NUM_TAKEOUT_QUEUE = 5  # Number of people that can be queued for takeout before balk
NUM_DINEIN_QUEUE = 5  # Number of people that can be queued for dine-in before balk


# Probabilities parameters
DINE_IN_PROB = 0.5  # Probability that a customer will dine in
TAKEOUT_PROB = 1 - DINE_IN_PROB  # Probability that a customer will order takeout

# Time Parameters
SIMULATION_TIME = 480  # Total simulation time in minutes (e.g., 8 hours)
MIN_PATIENCE = 1  # Minimum time a customer will wait before leaving
MAX_PATIENCE = 2  # Maximum time a customer will wait before leaving
MIN_SEATING_TIME = 1  # Minimum time a customer will spend eating
MAX_SEATING_TIME = 5  # Maximum time a customer will spend eating
MAX_QUEUE_LENGTH = 5  # Maximum number of customers that can be in the queue before balking
DINEIN_SERVICE_TIME = 5  # Average service time for dine-in customers (e.g., 5 minutes)
TAKEOUT_SERVICE_TIME = 3  # Average service time for takeout customers (e.g., 3 minutes)
PAYMENT_TIME = 1  # Average time for a customer to pay their bill (e.g., 2 minutes)


# rate parameters
ARRIVAL_RATE = 1 / 5  # Average arrival rate for dine-in (e.g., 1 customer every 5 minutes)
BASE_SERVICE_RATE = 1 / 4  # Base service rate for dine-in customers (e.g., service time of 4 minutes)
TAKEOUT_SERVICE_RATE = 1 / 3  # Service rate for takeout orders (e.g., service time of 3 minutes)
KITCHEN_COOK_RATE = 1 / 10  # Average time to cook an order (e.g., 10 minutes per order)

# Random Seed
RANDOM_SEED = 42

OBTAIN_REPRODUCIBLE_RESULTS = True

# dictionary of simulation parameters
sim_params = {
    'NUM_SERVERS': NUM_SERVERS,
    'NUM_COOKS': NUM_COOKS,
    'NUM_TABLES': NUM_TABLES,
    'NUM_TAKEOUT_QUEUE': NUM_TAKEOUT_QUEUE,
    'NUM_DINEIN_QUEUE': NUM_DINEIN_QUEUE,
    'DINE_IN_PROB': DINE_IN_PROB,
    'TAKEOUT_PROB': TAKEOUT_PROB,
    'SIMULATION_TIME': SIMULATION_TIME,
    'MIN_PATIENCE': MIN_PATIENCE,
    'MAX_PATIENCE': MAX_PATIENCE,
    'MIN_SEATING_TIME': MIN_SEATING_TIME,
    'MAX_SEATING_TIME': MAX_SEATING_TIME,
    'MAX_QUEUE_LENGTH': MAX_QUEUE_LENGTH,
    'DINEIN_SERVICE_TIME': DINEIN_SERVICE_TIME,
    'ARRIVAL_RATE': ARRIVAL_RATE,
    'BASE_SERVICE_RATE': BASE_SERVICE_RATE,
    'TAKEOUT_SERVICE_RATE': TAKEOUT_SERVICE_RATE,
    'KITCHEN_COOK_RATE': KITCHEN_COOK_RATE,
    'RANDOM_SEED': RANDOM_SEED,
    'PAYMENT_TIME': PAYMENT_TIME,
    'TAKEOUT_SERVICE_TIME': TAKEOUT_SERVICE_TIME,
    'OBTAIN_REPRODUCIBLE_RESULTS': OBTAIN_REPRODUCIBLE_RESULTS
}

In [13]:
env = simpy.Environment()

# implement FIFO queue to hold caseid values
caseid_queue_dinein = queue.Queue()
caseid_queue_takeout = queue.Queue()

# set up limited resource: Tables, Waiters, Cooks

# tables are a limited resource
available_tables = simpy.Resource(env, capacity = NUM_TABLES)

# servers are a limited resource
available_servers = simpy.Resource(env, capacity = NUM_SERVERS)

# cooks are a limited resource
available_cooks = simpy.Resource(env, capacity = NUM_COOKS)

# dictionary of resources
resources = {
    'tables': available_tables,
    'servers': available_servers,
    'cooks': available_cooks
}


caseid = -1  # dummy caseid for beginning of simulation
# dummy caseid values will be omitted from the event_log
# prior to analyzing simulation results

# beginning record in event_log list of tuples of the form
# form of the event_log tuple item: (caseid, time, activity)

event_log = [(caseid,0,'null_start_simulation')]

# event_monitor = partial(event_monitor, event_log)
env.process(event_log_append(env, caseid, env.now, 'start_simulation', event_log))
 
# call customer arrival process/generator to begin the simulation
env.process(arrival_waiting(env, sim_params, resources, caseid_queue_dinein, caseid_queue_takeout, event_log))  

env.run(until = sim_params['SIMULATION_TIME'])  # start simulation with monitoring and fixed end-time

NEXT ARRIVAL TIME:  31
Customer joins takeout queue caseid = 1 time = 31 queue_length = 1
NEXT ARRIVAL TIME:  67
Takeout order being prepared caseid = 1 time = 31 wait = 0
Takeout order finished caseid = 1 time = 34
Customer joins takeout queue caseid = 2 time = 67 queue_length = 1
NEXT ARRIVAL TIME:  80
Takeout order being prepared caseid = 2 time = 67 wait = 0
Takeout order finished caseid = 2 time = 70
Customer joins dine-in queue caseid = 3 time = 80 queue_length = 1
NEXT ARRIVAL TIME:  94
Customer seated and served caseid = 3 time = 80 wait = 0
Customer finished eating caseid = 3 time = 85
Customer joins takeout queue caseid = 4 time = 94 queue_length = 1
NEXT ARRIVAL TIME:  97
Takeout order being prepared caseid = 4 time = 94 wait = 0
Takeout order finished caseid = 4 time = 97
Customer joins takeout queue caseid = 5 time = 97 queue_length = 1
NEXT ARRIVAL TIME:  118
Takeout order being prepared caseid = 5 time = 97 wait = 0
Takeout order finished caseid = 5 time = 100
Customer j

In [14]:
# total number of customers served as maximum caseid value
total_customers = max([item[0] for item in event_log])

# total number of dine-in customers
dinein_customers = 0
for item in event_log:
    if item[2] == 'join_queue_dinein':
        dinein_customers += 1

# total number of takeout customers
takeout_customers = 0
for item in event_log:
    if item[2] == 'join_queue_takeout':
        takeout_customers += 1

# total number of customers that left due to impatience
impatience = 0
for item in event_log:
    if item[2] == 'impatience':
        impatience += 1

# total number of customers that left due to balking
balk = 0
for item in event_log:
    if item[2] == 'balk':
        balk += 1

# total number of customers that finished eating
finished_eating = 0
for item in event_log:
    if item[2] == 'finished_eating':
        finished_eating += 1

# print out the results of the simulation
print("Total Customers Served: ", total_customers)
print("Dine-in Customers: ", dinein_customers)
print("Takeout Customers: ", takeout_customers)
print("Customers Left Due to Impatience: ", impatience)
print("Customers Balked: ", balk)
print("Customers Finished Eating: ", finished_eating)

Total Customers Served:  45
Dine-in Customers:  24
Takeout Customers:  21
Customers Left Due to Impatience:  7
Customers Balked:  0
Customers Finished Eating:  17


In [21]:
# create a pandas dataframe from the event log
event_log_df = pd.DataFrame(event_log, columns = ['caseid', 'time', 'activity'])


# remove the first 2 rows of the event log
event_log_df = event_log_df.iloc[2:]

# pivot the event log dataframe
event_log_pivot = event_log_df.pivot(index = 'caseid', columns = 'activity', values = 'time')

# print out the pivoted event log
event_log_pivot

# add columns for missing activities if they do not exist
all_columns = ['arrival', 'join_queue_dinein', 'join_queue_takeout', 'seated', 'served', 'eating', 'finished_eating', 'paying', 'leaving', 'balk', 'impatience']

for column in all_columns:
    if column not in event_log_pivot.columns:
        event_log_pivot[column] = np.nan

# reorder the columns
# arrival - 1, join_queue_dinein - 2, join_queue_takeout - 3, seated - 4, served - 5, eating - 6, finished_eating - 7, paying - 8, leaving - 9, balk - 10, impatience - 11
event_log_pivot = event_log_pivot[['arrival', 'join_queue_dinein', 'join_queue_takeout', 'seated', 'served', 'eating', 'finished_eating', 'paying', 'leaving', 'balk', 'impatience']]

# print out the reordered pivoted event log
event_log_pivot

activity,arrival,join_queue_dinein,join_queue_takeout,seated,served,...,finished_eating,paying,leaving,balk,impatience
caseid,,,,,,,,,,,
1,31.0,NaN,31.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2,67.0,NaN,67.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
3,80.0,80.0,NaN,80.0,80.0,...,85.0,85.0,86.0,NaN,NaN
4,94.0,NaN,94.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
5,97.0,NaN,97.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
41,428.0,428.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,429.416575
42,430.0,430.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,431.269421
43,430.0,NaN,430.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [ ]:
# calculate the time spent in the restaurant for each customer
event_log_pivot['time_in_system'] = event_log_pivot['leaving'] - event_log_pivot['arrival']

# calculate the time spent eating for each customer
event_log_pivot['time_eating'] = event_log_pivot['finished_eating'] - event_log_pivot['eating']

# calculate the time spent waiting for each customer
# 'seated' - 'join_queue_dinein'
# 'seated' - 'join_queue_takeout'
event_log_pivot['time_waiting'] = event_log_pivot['seated'] - event_log_pivot['join_queue_dinein']
event_log_pivot['time_waiting'] = event_log_pivot['time_waiting'].fillna(event_log_pivot['seated'] - event_log_pivot['join_queue_takeout'])